In [1]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from sklearn import preprocessing

import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import random
import math as math
import re

MAX_ENTROPY = 0
MAX_LENGTH = 0


In [2]:
!pip install --upgrade pip
!pip install scikit-learn --upgrade

In [3]:
# loading the data frame
def load_dataframe(file):
    df = pd.read_csv(file, error_bad_lines = False, warn_bad_lines=False)
    df = shuffle(df)
    return df


In [4]:
# load train data and test data for Naive Bayes

def NB_load_training_data(df, strength_class, train_size):
    # Strength - three values(0 , 1 , 2) i.e. 0 for weak, 1 for medium, 2 for strong.
    # 3 types of password 0,1,2
    df_train = df.loc[df['strength'].isin(strength_class)]
    df_train = df_train.head(train_size)
    return df_train


def NB_load_test_data(df, strength_class, test_size_start, test_size_end):
    #Strength - three values(0 , 1 , 2) i.e. 0 for weak, 1 for medium, 2 for strong.
    # 3 types of password 0,1,2
    df_test = df.loc[df['strength'].isin(strength_class)]
    df_test = df_test.iloc[test_size_start:test_size_end]

    return df_test

In [5]:
train_size = 10000
classes = [0,1,2]
test_size = 5000


df = load_dataframe("data.csv")
print(len(df))
df_train = NB_load_training_data(df,classes, train_size)
df_test = NB_load_test_data(df , classes, train_size + 1,train_size + 1 + test_size)

/var/folders/h7/3bzjl2hs4178tvwhq5sftr840000gn/T/ipykernel_1814/1506715142.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file, error_bad_lines = False, warn_bad_lines=False)
/var/folders/h7/3bzjl2hs4178tvwhq5sftr840000gn/T/ipykernel_1814/1506715142.py:3: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file, error_bad_lines = False, warn_bad_lines=False)


669640


In [6]:
#Sklearn Naive Bayes Model
number = preprocessing.LabelEncoder()

df_train_sk = df_train.copy()
df_test_sk = df_test.copy()

# train
df_train_sk['password'] = number.fit_transform(df_train_sk.password)
X = df_train_sk['password']
X = np.asarray(X)
X_train = X.reshape(-1, 1)
y_train = df_train_sk['strength']
# test
df_test_sk['password'] = number.fit_transform(df_test_sk.password)
X = df_test_sk['password']
X = np.asarray(X)
X_test = X.reshape(-1, 1)
y_test = df_test_sk['strength']


gnb = GaussianNB()



In [7]:
# Naive Bayes From Scratch 
class NaiveBayes:
   
    def __init__(self, df, classes, train_size):
        self.df = df
        self.classes = classes
        self.train_size = train_size
        self.password_class_letter_dic_list = []
        self.password_class_letters_list = []
        self.password_class_p = []
        self.all_letter_dic = {}
        self.total_letters = 0
        self.password_class_p = []
     

        for r in range(0,len(classes)):
            password_dic,password_total_letters = self.letter_dic_generator(classes[r])
            self.password_class_letter_dic_list.append(password_dic)
            self.password_class_letters_list.append(password_total_letters)

        self.all_letter_dic, self.total_letters = self.letter_dic_generator()
        
        for i in range(0,len(classes)):
            self.password_class_p.append(self.password_type_counter(df,classes[i]))

    def password_type_counter(self,df,strength):
        return df.loc[df['strength'] == strength]['strength'].count()
    
    def letter_dic_generator(self,dic_strength = -1):
        letter_dic = {}
        total_letters = 0
        for i in range(0, len(self.df)):
            password = self.df.iloc[i]['password']
            strength = str(self.df.iloc[i]['strength'])
            if(strength == str(dic_strength) or dic_strength == -1):
                for letter in password:
                    if(letter in letter_dic):
                        letter_dic[letter] += 1
                    else:
                        letter_dic[letter] = 1 
                    total_letters += 1
        return (letter_dic, total_letters)

    def denominator_calc(self, password):
        p_x = 0
        for r in range(0,len(self.classes)):
            y = 1 
            for letter in password:
                y = y  * (self.password_class_letter_dic_list[r][letter]/self.password_class_letters_list[r])
            y = y * self.password_class_p[r]
            p_x = y + p_x
        return p_x
        
    def probability_calc (self, p,dic, total_letters_counter, password):
        p_x = self.denominator_calc(password)
        p_x_y = 1
        for letter in password:
            p_x_y = p_x_y * (dic[letter]/total_letters_counter)
        p_y_x = p_x_y * p / p_x
        return p_y_x

    def password_strength_checker(self, password):
        prob = []
        for r in range(0,len(self.classes)):
            p = self.probability_calc(self.password_class_p[r],self.password_class_letter_dic_list[r],self.password_class_letters_list[r],password)
            prob.append(p)
        
        max_value = max(prob)
        max_index = prob.index(max_value)
        return self.classes[max_index]
    
    def accuracy(self,df_test):
        df_test_num_counter = 0
        accuracy = 0 
        for i in range(0, len(df_test)):
            try:
                password = df_test.iloc[i]['password']
                strength = str(df_test.iloc[i]['strength'])
                output = self.password_strength_checker(password)
                if(str(output) == strength):
                    accuracy = accuracy + 1
                df_test_num_counter += 1
            except:
                continue
        
        return (accuracy/df_test_num_counter)
    

In [8]:
# Comparing the two Naive Bates Models models
NaiveBayesClass = NaiveBayes(df_train, classes, train_size)
y_pred = gnb.fit(X_train, y_train).predict(X_test)

accuracy = ((y_test == y_pred).sum())/test_size

print("Accuracy of Scratch Naive Bayes:" + str(NaiveBayesClass.accuracy(df_test)))
print("Accuracy of Sklearn Naive Bayes model is: %f " % (accuracy))
#print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))


Accuracy of Scratch Naive Bayes:0.8285136501516683
Accuracy of Sklearn Naive Bayes model is: 0.728000 


In [9]:
def entropy(string):
    "Calculates the Shannon entropy of a string"

    uppercase = False
    lowercase = False
    digit = False
    specialchar = False
    for char in string:
        k = char.islower() 
        c = char.isupper() 
        if(k == True ):
             lowercase = True
        elif(c==True):
            uppercase = True
        elif(char.isnumeric()):
            digit = True
        else:
            specialchar = True
    p = 0
    if(uppercase):
        p = p + 26
    if(lowercase):
        p = p + 26
    if(digit):
        p = p + 10
    if(specialchar):
        p = p + 30
    entropy = (math.log(p**len(string)) / math.log(2.0))
    return entropy

#https://generatepasswords.org/how-to-calculate-entropy/


In [10]:
def KMeans_load_train_data(df, strength_class, train_size):
    df_train = df.head(train_size)
    df_train = df_train.drop('strength', axis=1)
    
    password_length_ary = []
    entropy_list = []
    global MAX_ENTROPY 
    MAX_ENTROPY = 0
    global MAX_LENGTH 
    MAX_LENGTH = 0 
    for passwordIndex in range(0,train_size):
        password = df_train.iloc[passwordIndex]['password']
        if(entropy(password) > MAX_ENTROPY):
            MAX_ENTROPY =  entropy(password) 
        if(len(password) > MAX_LENGTH):
            MAX_LENGTH = len(password)
    
    for passwordIndex in range(0,train_size):
        password = df_train.iloc[passwordIndex]['password']
        
        password_length_ary.append(len(password)/MAX_LENGTH)
        entropy_list.append(entropy(password)/MAX_ENTROPY)
    
    df_train.insert(1, 'password_length',password_length_ary)
    df_train.insert(2,'entropy',entropy_list)
    return df_train

def KMeans_load_test_data(df, strength_class, test_size_start, test_size_end):
    df_test_temp = df.iloc[test_size_start:test_size_end]
    df_test = df_test_temp.loc[df['strength'].isin(strength_class)]

    password_length_ary = []
    entropy_list = []
    
    for passwordIndex in range(0,len(df_test)):
        password = df_test.iloc[passwordIndex]['password']
        password_length_ary.append(len(password)/MAX_LENGTH)
        entropy_list.append(entropy(password)/MAX_ENTROPY)
        
    df_test.insert(1, 'password_length',password_length_ary)
    df_test.insert(2,'entropy',entropy_list)

    return df_test


def KMeans_sklearn_data_generator(df):
    KMeans_sklearn_data = []
    for r in range(0,len(df)):
        password = df.iloc[r]
        point = [float(password['password_length']),float(password['entropy'])]
        KMeans_sklearn_data.append(point)
    return  (KMeans_sklearn_data)


In [11]:
class KMeansClass:
    def __init__(self, df_train,num_classes,num_features):
        self.df_train = df_train
        self.num_classes = num_classes
        self.final_centroids = []
        self.num_features = num_features
    
        self.centroids_cluster_counter = [0] * num_classes
        
    def intial_centriod_generator(self):
        distances = []
        intial_centroids = []
        rand_values = []
  
        for r in range(0, self.num_classes):
            while(True):
                random_index = random.randint(0, len(df_train)-1)
                if(random_index not in rand_values):
                    rand_centroid = self.df_train.iloc[random_index]
                    point = []
                    point.append(float(rand_centroid['password_length']))
                    point.append(float(rand_centroid['entropy']))
                    
                    if(point not in intial_centroids):
                        intial_centroids.append(point)
                        rand_values.append(random_index)
                        break
        
        intial_centroids = sorted(intial_centroids, key=sum)
        return intial_centroids
    
    def model_generator(self, max_itteration, centroids):
        number_itteration = 0
        group = []

        while(True):
            number_itteration = number_itteration + 1
            temp_group = group
            group = []
            centroids_itter_sum = [[0,0],[0,0],[0,0]] 
            centroids_points_sum_counter = [0,0,0]

            for passwordIndex in range(0,len(df_train)):
                distances = []
                
                for centroid_counter in range(len(centroids)):
                    password = df_train.iloc[passwordIndex]
                    distance_calc = math.sqrt(((float(password['password_length'])-float(centroids[centroid_counter][0]))**2) +((float(password['entropy'])-float(centroids[centroid_counter][1]))**2))
                    distances.append([distance_calc,centroid_counter])
                
                min_diststance = min(distances, key=lambda x: x[0])

                centroids_points_sum_counter[min_diststance[1]] += 1
                                               
                centroids_itter_sum[min_diststance[1]][0] += password['password_length']
                centroids_itter_sum[min_diststance[1]][1] += password['entropy']
                
                group.append(min_diststance[1])

            
            for centroid_counter in range(0,self.num_classes):
                new_point = []
                new_point.append(centroids_itter_sum[centroid_counter][0] / centroids_points_sum_counter[centroid_counter])
                new_point.append(centroids_itter_sum[centroid_counter][1] / centroids_points_sum_counter[centroid_counter])

                centroids[centroid_counter] = new_point
                

            if(number_itteration >= max_itteration):
                change = False
                df_train.insert(self.num_features , 'predicted_strength',group)
                return centroids
            if(temp_group == group):
                change = False
                df_train.insert(self.num_features , 'predicted_strength',group)
                
                return centroids

    def strength_pred(self,password):
        distances = []
        for r in range(len(self.final_centroids)):
            distance_calc = math.sqrt((float(password['password_length'])-self.final_centroids[r][0])**2 +(float(password['entropy'])-self.final_centroids[r][1])**2)
            distances.append([distance_calc,r])
        min_diststance = min(distances, key=lambda x: x[0])
        pred_strength = min_diststance[1] 
        return pred_strength

    def accuracy(self,df_test,test_size):
        accuracy = 0
        for passwordIndex in range(0,test_size):
            password = df_test.iloc[passwordIndex]
            #print(self.strength_pred(password))
            if(self.strength_pred(password) == password['strength']):
                accuracy = accuracy + 1
        return accuracy/test_size
        
    def fit(self, max_itteration):
        centroids = self.intial_centriod_generator()
        self.final_centroids = self.model_generator(max_itteration,centroids)
        

In [12]:


train_size = 50
classes = [0,1,2]
test_size = 1000
max_itteration = 500

df_train = KMeans_load_train_data(df,classes, train_size)
df_test = KMeans_load_test_data(df , classes, train_size + 1,train_size + 1 + test_size)



KMeansModel= KMeansClass(df_train, len(classes), 3)
KMeansModel.fit(max_itteration)
#print(KMeansModel.final_centroids)
print(KMeansModel.accuracy(df_test,test_size))



X_train_KMeans_sklearn = KMeans_sklearn_data_generator(df_train)

kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(X_train_KMeans_sklearn)
#print(kmeans.cluster_centers_)

X_test = KMeans_sklearn_data_generator(df_test)
pred_kmeans_sklearn = (kmeans.predict(X_test))



0.442


In [13]:
df_train.head()

,password,password_length,entropy,predicted_strength
27058,epwdlbxt03,0.50,0.427663,0
160372,009tf5ka0w,0.50,0.427663,0
332061,mehdiaziz123,0.60,0.513195,1
182028,fecil30,0.35,0.299364,0
390496,goss88,0.30,0.256598,0


In [14]:
def NN_load_train_data(df, strength_class, train_size):
    df_train = df.head(train_size)
    
    password_length_ary = []
    entropy_list = []
    
    global MAX_ENTROPY
    MAX_ENTROPY = 0
    global MAX_LENGTH 
    MAX_LENGTH = 0 
    
    for passwordIndex in range(0,len(df_train)):
        password = df_train.iloc[passwordIndex]['password']
        if(entropy(password) > MAX_ENTROPY):
            MAX_ENTROPY =  entropy(password) 
        if(len(password) > MAX_LENGTH):
            MAX_LENGTH = len(password)
            
    for passwordIndex in range(0,train_size):
        password = df_train.iloc[passwordIndex]['password']
        
        password_length_ary.append(len(password)/MAX_LENGTH)
        entropy_list.append(entropy(password)/MAX_ENTROPY)
    
    df_train.insert(1, 'password_length',password_length_ary)
    df_train.insert(2,'entropy',entropy_list)
    
    return df_train

def NN_load_test_data(df, strength_class, test_size_start, test_size_end):
    df_test_temp = df.iloc[test_size_start:test_size_end]
    df_test = df_test_temp.loc[df['strength'].isin(strength_class)]

    password_length_ary = []
    entropy_list = []
    
    for passwordIndex in range(0,len(df_test)):
        password = df_test.iloc[passwordIndex]['password']
        password_length_ary.append(len(password)/MAX_LENGTH)
        entropy_list.append(entropy(password)/MAX_ENTROPY)
        
    df_test.insert(1, 'password_length',password_length_ary)
    df_test.insert(2,'entropy',entropy_list)

    return df_test

In [18]:
class NNClass:
    def __init__(self):
        self.NN = None
        #self.NNDef = None
        
    def model_generator(self,df_train,neuron_num_layer1,neuron_num_layer2):
    
        columns = ["password_length","entropy"]
        y_train = df_train["strength"]
        df_train = df_train.drop(["strength"], axis=1)

        df_train['password'] = number.fit_transform(df_train.password)

        for feature in columns:
            le =  preprocessing.LabelEncoder()
            df_train[feature] = le.fit_transform(df_train[feature])
        X_train = df_train.to_numpy()
        
        self.NN =  MLPClassifier(hidden_layer_sizes=(neuron_num_layer1,neuron_num_layer2))
        #self.NNDef =  MLPClassifier()
        self.NN = self.NN.fit(X_train,y_train)
        #self.NNDef = self.NNDef.fit(X_train,y_train)
            
    def accuracy(self, df_test):
        df_test['password'] = number.fit_transform(df_test.password)
        y_test = df_test["strength"]
        df_test = df_test.drop(["strength"], axis=1)
        columns = ["password_length","entropy"]
        for feature in columns:
            le =  preprocessing.LabelEncoder()
            df_test[feature] = le.fit_transform(df_test[feature])
        X_test = df_test.to_numpy() 
        y_pred = self.NN.predict(X_test)
        
        #y_predDef = self.NNDef.predict(X_test)
            
        accuracy = accuracy_score(y_test,y_pred) * 100
        #accuracyDef = accuracy_score(y_test,y_predDef) * 100
        #print("Default Accuracy")
        #print(accuracyDef)
        return accuracy

In [21]:

train_size = 2000
df_train = NN_load_train_data(df,classes, train_size)
test_size = 1000
df_test = NN_load_test_data(df , classes, train_size+1,train_size+1+test_size)

accuracy_list = []
neuron_list = []
password_dic_length = {}
entropy_dic = {}
for r in range(0,len(df_train)):
    password  = df_train.iloc[r]
    if(password["entropy"] not in entropy_dic ):
        entropy_dic[password["entropy"]] =password["entropy"]
    if(len(password["password"]) not in password_dic_length):
        password_dic_length[len(password["password"])] = len(password["password"])

#print(len(entropy_dic))
#print(len(password_dic_length))
                                                             
NNModel = NNClass()

for layer1_neruon_count in range(1,101,20):
    for layer2_neruon_count in range(1,101,20):
        total = 0
        trial_num = 8
        for r in range(0,trial_num):
            NNModel.model_generator(df_train,len(entropy_dic),len(password_dic_length))
            total = total+ NNModel.accuracy(df_test)
        acr = total/trial_num
        neuron_list.append([layer1_neruon_count,layer2_neruon_count])
        accuracy_list.append(acr)
        #print(NNModel.accuracy(df_test))




/Users/samangivian/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/samangivian/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/samangivian/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/samangivian/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:
max_acr = (max(accuracy_list))

print("Accuracy list for different number of neurons:")
print(accuracy_list)
max_index = accuracy_list.index(max_acr)
print("Maximum accuracy")
print(max_acr)
print("Number of neurons in each layer of the maximum accuracy:")
print(neuron_list[max_index])

Accuracy list for different number of neurons:
[83.275, 83.125, 84.60000000000001, 86.3875, 85.175, 82.4125, 85.18750000000001, 84.43749999999999, 82.27499999999999, 85.5625, 84.3375, 84.23750000000001, 83.8, 84.32500000000002, 84.8, 85.8125, 85.625, 85.49999999999999, 84.75, 87.05000000000001, 85.94999999999999, 83.83749999999999, 83.225, 81.8625, 84.46249999999999]
Maximum accuracy
87.05000000000001
Number of neurons in each layer of the maximum accuracy:
[61, 81]
